# Importing libraries

In [1]:
import pandas as pd
import plotly   
import plotly.express as px
import plotly.io as pio

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State 

app = dash.Dash(__name__)

# Loading data

In [2]:
covid_Data_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
covid = pd.read_csv(covid_Data_url)
covid.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,31238,31517,31836,32022,32324,32672,32951,33190,33384,33594
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2466,2535,2580,2662,2752,2819,2893,2964,3038,3106
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,13571,13907,14272,14657,15070,15500,15941,16404,16879,17348
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,276,284,291,315,328,346,346,346,386,386


In [3]:
coviddrop = covid.drop(['Province/State','Lat','Long'],axis =1)
coviddrop

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,31238,31517,31836,32022,32324,32672,32951,33190,33384,33594
1,Albania,0,0,0,0,0,0,0,0,0,...,2466,2535,2580,2662,2752,2819,2893,2964,3038,3106
2,Algeria,0,0,0,0,0,0,0,0,0,...,13571,13907,14272,14657,15070,15500,15941,16404,16879,17348
3,Andorra,0,0,0,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
4,Angola,0,0,0,0,0,0,0,0,0,...,276,284,291,315,328,346,346,346,386,386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Sao Tome and Principe,0,0,0,0,0,0,0,0,0,...,713,714,715,717,719,719,720,721,724,724
262,Yemen,0,0,0,0,0,0,0,0,0,...,1128,1158,1190,1221,1240,1248,1265,1284,1297,1318
263,Comoros,0,0,0,0,0,0,0,0,0,...,272,303,303,303,309,309,311,311,311,313
264,Tajikistan,0,0,0,0,0,0,0,0,0,...,5900,5900,6005,6058,6058,6159,6213,6262,6315,6364


# Transforming and grouping data

In [4]:
Covidtransform = coviddrop.melt(id_vars = 'Country/Region')
Covidtransform.columns= ['Country','Date','No of Cases']
Covidtransform['Date'] = pd.to_datetime(Covidtransform.Date)
Covidtransform
Covidgroup = Covidtransform.groupby([pd.Grouper(key = 'Date', freq = 'M'),'Country'])['No of Cases'].sum().reset_index().sort_values(by = 'No of Cases',ascending = False)
covidGroupfilter = Covidgroup[Covidgroup['Date'] <= '2020-06-30']
covidGroupfilter

,Date,Country,No of Cases
1114,2020-06-30,US,64933225
926,2020-05-31,US,45406912
963,2020-06-30,Brazil,28161222
738,2020-04-30,US,19552396
1080,2020-06-30,Russia,16097579
...,...,...,...
217,2020-02-29,Cabo Verde,0
219,2020-02-29,Cameroon,0
221,2020-02-29,Central African Republic,0
222,2020-02-29,Chad,0


## Adding layout to the dashboard

In [5]:
app.layout = html.Div([
     # Header 
        html.H1("COVID-19 Data Analysis", style={"textAlign": "center"}),

  
    
    html.Div([

        html.Br(),
        dcc.Tabs(id="tabs", children=[
        # Defining the layout of the first Tab
        dcc.Tab(label='COVID Trends', children=[
            html.Div([
                html.H1("Monthly COVID 19 Cases", 
                        style={'textAlign': 'center'}),
                html.Label(['Choose no of Countries to Compare:'],style={'font-weight': 'bold', "text-align": "center",'font-size':'20px'}),
         # Dropdown 
        dcc.Dropdown(id='CountryName',
            options=[{'label':x, 'value':x} for x in covidGroupfilter.sort_values('Country')['Country'].unique()],
            value=['US','Italy','Australia'],
            multi=True,
            disabled=False,
            clearable=True,
            searchable=True,
            style={"display": "block", "margin-left": "auto", 
                                    "margin-right": "auto", "width": "100%"},
            placeholder='Choose Country...',
            persistence='string',
            persistence_type='memory'),
            dcc.Graph(id='linegraph')
    ]
        ,className='container'
    )]),
            # Defining the layout of the second tab
        dcc.Tab(label='COVID Daily Trend', children=[
            html.H1("Daily trend", 
                    style={"textAlign": "center"})
        ]),
        ])])])

# Defining function and callback

In [6]:
@app.callback(
    Output('linegraph','figure'),
    [Input('CountryName','value')
    ]
)


def build_graph(First_Country):
    dff1 = covidGroupfilter.loc[covidGroupfilter['Country'].isin(First_Country)]
    fig = px.line(dff1, x="Date", y="No of Cases", color='Country', height=600)
    fig.update_layout(yaxis={'title':'Known Cases'},
                      title={'text':'Biweekly COVID 19 Cases',
                      'font':{'size':28},'x':0.5,'xanchor':'center'},
                      # hover code  
                      hovermode='x unified',
                    # Log and linear code
                        updatemenus=[
                            dict(
                                type = "buttons",
                                direction = "left",
                                buttons=list([
                            dict(
                                args=[{"yaxis.type": "linear"}],
                                label="LINEAR",
                                method="relayout"
                                ),
                            dict(
                                args=[{"yaxis.type": "log"}],
                                label="LOG",
                                method="relayout"
                )
            ]),
        ),
    ],
                       # 1M, 1Y tab code
                       xaxis=dict(
                            rangeselector=dict(
                            buttons=list([
                                dict(count=1,
                                 label="1M",
                                 step="month",
                                 stepmode="backward"),
                                dict(count=6,
                                     label="6M",
                                     step="month",
                                     stepmode="backward"),
                                dict(count=1,
                                     label="YTD",
                                     step="year",
                                     stepmode="todate"),
                                dict(count=1,
                                     label="1Y",
                                     step="year",
                                     stepmode="backward"),
                                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
         #setting default background             
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
                     )  
    return fig

In [7]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jul/2020 12:08:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 12:08:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 12:08:29] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 12:08:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 13:09:00] "POST /_dash-update-component HTTP/1.1" 200 -
